In [ ]:
#| default_exp power


# Direct Power Analysis

Good grief this is complicated

This section includes the formula for

- Load coefficents J-9 to 11 and 14, 25,26.
- torque coefficient in trial conditions J-12
- propeller advance coefficient J-13 and J-24. The quadratic formula
- propellor efficiency in trial condition J-15 and J-27
- speed of flow into properller J-16
- fullscale wake fraction in trial conditions J-17
- load factor in trial conditions J-18
- Total resistance in trial conditions J-19
- Total resistance in ideal conditions J-22
- scale corellation factor J-21
- load factor idea J-23
- propeller shaft speed -J-29


Some things should come with a bit more clarification e.g. 'wake-fraction', what is it?

In [ ]:
#| export
import numpy as np
from fastcore.test import *

### Correction delivered power

Where does $\Delta R$ get calculated

$$\Delta P = \frac{\Delta R V_s}{\eta_{Dms}}+ P_{Dms}(1-\frac{\eta_{Dms}}{\eta_{Did}}) $$

In [ ]:
#| export
def correction_delivered_power(
    p_dms:float, #delivered power [W]
    resistance_increase:float, #Resistance increase derived from data measured in seatrial
    stw:float, #speed through water [m/s]
    eta_id:float, #propulsive efficiency in the ideal conditions
    eta_ms:float, # propulsive efficiency in the seattrial

)-> float:
    
    "calculates the corrected delivered power, used as part of the direct power analysis"
    
    return resistance_increase * stw /eta_id + p_dms * (1- eta_ms/eta_id)
    

Example of using function, generally this is not used directly

In [ ]:
correction_delivered_power(1e4, 1e3, 10, 0.8, 0.7)

13750.000000000002

### Propulsive efficiency correction
This can be used to calculate both the ideal and trial conditions reffered to in equations J-3 and J-5
$$\eta_D = \eta_o \eta_R \frac{1-t}{1-w_s} $$


In [ ]:
#| export
def propulsive_efficiency_corr(n_o:float, #open water efficiency
                          n_r:float, #relative rotative efficiency
                          t:float, #thrust deduction factor
                         w_s:float, #full-scale wake fraction
                         ):
    "Calculates propeller efficiency adjusting for additional resistances"
    return n_o*n_r*(1-t)/(1-w_s)
    

In [ ]:
propulsive_efficiency_corr(0.58, 0.7, 0.1, 0.5)

0.7308

In [ ]:
#| hide

test_eq(propulsive_efficiency_corr(0.58, 0.7, 1, 0),0)
test_eq(propulsive_efficiency_corr(1, 1, 0, 0.5),2)
#checks to make sure there is a divide by 0 error
test_fail(lambda: propulsive_efficiency_corr(0.58, 0.7, 0.0, 1),  contains="division by zero" )


### Full scale wake fraction
Equation J-4 and J-20
$$w_s =  1- (1- w_M)e_i$$


In [ ]:
#| export
def full_scale_wake_fraction(wake_fraction_model:float,
                            scale_correlation_factor:float
                            )-> float:
    
    "used to scale from model results to full-scale vessel"
    
    return 1- (1- wake_fraction_model) * scale_correlation_factor

In [ ]:
full_scale_wake_fraction(0.4, 0.8)

0.52

In [ ]:
#| hide

test_eq(full_scale_wake_fraction(1, 0.8), 1)
test_close(full_scale_wake_fraction(0, 0.8), 0.2, eps = 1e-6)

## Self propulsion factors

This function allows for ship factors to be adjust accounting for the difference between ideal/model conditions and the conditions during the trial. This function is equivalent to equation J-6 to J-8.
The function is essentially the following equation

$$x_{test} = x_{ideal} + \Delta x_R (\frac{\Delta R}{R_{ideal}}),$$
Where $x$ is the variable to be adjusted $x_{ideal}$ is the value of the variable under ideal conditions, $\Delta x_R$ the coefficient of of change for each unit of $\frac{\Delta R}{R_{ideal}}$, $\Delta R$ the resistance increase derived from data measured during the sea trial and $R_{ideal}$ is the resistance under ideal conditions. The values of $x_{ideal}$ are obtained from a model test. The value of $\Delta x_R$ is found by fitting a linear model using data gathered during a specific tank tests. The process of obtaining the value of $x_R$ is described in detail in section J.2.

In practice however, these sets of values are not needed as the deviations are often negligable in comparison to the variation of $\eta_O$. That using 
$$ x_{test} \approx x_{ideal},$$
is acceptable

In [ ]:
#| export
def self_propulsion_factors(
    x_ideal:float, #The variable in ideal conditions. It is acceptable to use this value without adjustments
    delta_x:float = 0, #The change per unit of the resistance ratios. Default is 0
    delta_r:float = 1, #increase in resistance from ideal conditions
    delta_r_ideal:float = 1 #Resistance in ideal conditions
) -> float:
    
    "Adjusting the self propulsion factors is only possible if the required model tests have been performed. By default this function returns the ideal value"
    
    return x_ideal + delta_x * (delta_r/delta_r_ideal)

Generally the adjustment for the self propulsion factors will be small. But if available should be applied

In [ ]:
self_propulsion_factors(0.8, 0.1, 1000, 10000)

0.81

In [ ]:
#| hide

test_eq(self_propulsion_factors(0.8), 0.8)


In [ ]:
import nbdev; nbdev.nbdev_export()